In [2]:
# Importing Libraries 
import pandas
import psycopg2

In [ ]:
# Connection eshtablishment 
conn = psycopg2.connect(dbname='postgres', user='postgres', password='postgres', host='localhost')

In [4]:
conn.autocommit = True
cur = conn.cursor()

cur.execute("""
    CREATE DATABASE data_silos
        LC_COLLATE='en-US'
        LC_CTYPE='en-US'
        TEMPLATE=template0;
""")